# Installation

In [9]:
# !pip install pythreejs
# !pip install numpy, scipy
# !pip install notebook==6.5.4
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

In [10]:
import numpy as np
import time

# Particle

In [11]:
class Particle:
    def __init__(self, pos, vel, mass):
        self.position = pos
        self.velocity = vel
        self.force = np.array([0, 0, 0])
        self.mass = mass
        
    def clear_force(self):
        self.force = np.array([0, 0, 0])

# Forces

In [ ]:
# Types of forces
# 1. Constant e.g. gravity
# 2. position dependent e.g. forces fields, winds
# 3. velocity dependent e.g. drag, friction
# 4. n-ary e.g. springs
# 5. collision

class Force:
    def apply(self, particles):
        pass

class Gravity(Force):
    def __init__(self, gravity=np.array([0, -9.8, 0])):
        self.G = gravity
        
    def apply(self, particles):
        for p in particles:
            p.force += p.mass * self.G

class Drag(Force):
    def __init__(self, k_drag=0.1):
        self.k_drag = k_drag
        
    def apply(self, particles):
        for p in particles:
            p.force += -self.k_drag * p.velocity

class Spring(Force):
    def __init__(self, particle1, particle2,  k_s, k_d, l0):
        self.p1 = particle1
        self.p2 = particle2
        self.k_s = k_s
        self.k_d = k_d
        self.l0 = l0
        
    def apply(self, particles):
        x1 = self.p1.position
        x2 = self.p2.position
        
        l = x1 - x2
        l_dot = self.p1.velocity - self.p2.velocity
        length = np.linalg.norm(l)
        
        f = -(self.k_s * (length - self.l0) + self.k_d * np.dot(l_dot, l) / length) * l / length
        
        self.p1.force += f
        self.p2.force += -f
        
class Collision(Force):
    pass

# Particle System

In [13]:
class ParticleSystem:
    def __init__(self, particles=None, forces=None,):
        
        self.particles = []
        self.forces = []
        
        
    def add_particle(self, particle):
        if isinstance(particle, list):
            self.particles.extend(particle)
            
        elif isinstance(particle, Particle):       
            self.particles.append(particle)
        
        else:
            raise ValueError("Invalid particle type")
        
    def add_force(self, force):
        if isinstance(force, list):
            self.forces.extend(force)
            
        elif isinstance(force, Force):       
            self.forces.append(force)
        
        else:
            raise ValueError("Invalid force type")
    
    def evaluate_derivative(self,):
        # (1) Loop over particles, zero force accumulators
        for p in self.particles:
            p.clear_force()
        
        # (2) Calculate forces by invoking apply functions, sum all forces into accumulators
        for f in self.forces:
            f.apply(self.particles)
        
    
    def render(self,):
        pass

# Numerical Method

In [14]:
class Integrator:
    def solve(self, particle_system, time_step):
        pass
    
class Euler(Integrator):
    def solve(self, particle_system, time_step):
        
        particle_system.evaluate_derivative()
        
        for p in particle_system.particles:
            a = p.force / p.mass
            p.velocity += a * time_step
            p.position += p.velocity * time_step
            
    def solve_adaptive(self, particle_system, init_time_step, tol):
        #TODO
        pass
            

class Midpoint(Integrator):
    def solve(self, particle_system, time_step):
        
        # Save initial position and velocity
        init_position = [p.position.copy() for p in particle_system.particles]
        init_velocity = [p.velocity.copy() for p in particle_system.particles]        
        
        particle_system.evaluate_derivative()
        
        # Compute midpoint position and velocity
        for p in particle_system.particles:
            a = p.force / p.mass
            p.velocity = p.velocity + a * time_step / 2
            p.position = p.position + p.velocity * time_step / 2
        
        # Compute forces at midpoint
        particle_system.evaluate_derivative()
        
        # Compute final position and velocity
        for i, p in enumerate(particle_system.particles):
            a = p.force / p.mass
            p.velocity = init_velocity[i] + a * time_step
            p.position = init_position[i] + p.velocity * time_step
        
class RK4(Integrator):
    def solve(self, particle_system, time_step):
        pass


# Render

In [ ]:
from pythreejs import * 
from IPython.display import display
import ipywidgets as widgetsf
from ipyevents import Event

camera = OrthographicCamera(
    left=-10, right=10, top=10, bottom=-10, near=0.1, far=100
)
camera.position = [0, 0, 10]  # Place camera in front of the scene.
camera.lookAt([0, 0, 0])

# Create a scene
scene = Scene()


# ----------------------------
# CREATE THE PARTICLE SYSTEM
# ----------------------------

# For our 2D particle system we generate a set of particles in the XY plane (with z=0)
num_particles = 50
positions = np.random.uniform(-5, 5, (num_particles, 3))
positions[:, 2] = 0  # Ensure particles lie in the XY plane

# Create a BufferGeometry with these positions.
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(positions, normalized=False)
    }
)

# Create a PointsMaterial to render the particles (red circles)
material = PointsMaterial(color='red', size=10.0)

# Create the Points object to add to the scene.
points = Points(geometry=geometry, material=material)
scene.add(points)


# ----------------------------
# SET UP THE RENDERER
# ----------------------------

renderer = Renderer(
    scene=scene, 
    camera=camera, 
    controls=[OrbitControls(controlling=camera, enableRotate=False)],
    width=600, height=600
)



/home/matthew/anaconda3/lib/python3.12/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


In [16]:
display(renderer)

Renderer(camera=OrthographicCamera(bottom=-10.0, far=100.0, left=-10.0, position=(0.0, 0.0, 10.0), projectionM…

In [41]:
import numpy as np
from pythreejs import *
from IPython.display import display
from ipywidgets import jslink, Output

# Debugging output
out = Output()

# Create multiple circles (meshes)
num_circles = 5
circles = []
pickers = []
colors = ['red', 'blue', 'green', 'yellow', 'purple']
positions = [[-1, 1, 0], [1, 1, 0], [-1, -1, 0], [1, -1, 0], [0, 0, 0]]

for i in range(num_circles):
    circle = Mesh(
        geometry=CircleGeometry(radius=0.2),
        material=MeshLambertMaterial(color=colors[i]),
        position=positions[i]
    )
    circles.append(circle)

# Scene setup
scene = Scene(children=circles + [AmbientLight()])
camera = OrthographicCamera(left=-2, right=2, top=2, bottom=-2, near=0.1, far=10, position=[0, 0, 5])
renderer = Renderer(camera=camera, scene=scene, controls=[])

display(renderer, out)  # Display both renderer and debug output widget

# Store selected circle
selected_circle = None  

# Function to track selected circle
def on_pick(change):
    global selected_circle
    selected_circle = change['owner']  # Each picker is attached to a different circle
    with out:
        print(f"Selected: {selected_circle.material.color}")

# Function to move the selected circle
def on_drag(change):
    global selected_circle
    if selected_circle:
        new_pos = change['new']
        selected_circle.position = [new_pos[0], new_pos[1], 0]  # Keep Z = 0 for 2D
        with out:
            print(f"Dragging {selected_circle.material.color} to {new_pos}")

# Attach a Picker to each circle
for circle in circles:
    picker = Picker(controlling=circle, event='mousedown')
    move_picker = Picker(controlling=circle, event='mousemove')
    
    picker.observe(on_pick, names=['object'])  # Select when clicked
    move_picker.observe(on_drag, names=['point'])  # Move the selected object
    
    renderer.controls.append(picker)
    renderer.controls.append(move_picker)
    
    pickers.append(picker)



Renderer(camera=OrthographicCamera(bottom=-2.0, far=10.0, left=-2.0, position=(0.0, 0.0, 5.0), projectionMatri…

Output()